In [1]:
import typing
import lzma
import os
import dataclasses
import itertools
import functools
from importlib import reload
from pathlib import Path
from types import MappingProxyType, SimpleNamespace

import cbor2
import attrs
import tqdm.auto
import jax
import jax.numpy as jnp
from flax import nnx
import optax
import jaxtyping as jt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from arc25 import symmetry, tools as arc25_tools
from arc25.symmetry import D4, transform_vector
from arc25 import serialisation
from arc25.dsl.types import Vector, Dir4
from arc25.vision2.symrep import SymDecompBase, SplitSymDecomp, SymDecompDims, standard_rep, RepSpec
from arc25.vision2.fields import FieldDims, CoordinateGrid
from arc25.vision2.linear import SpaceSymmetricLinear, SpaceSymmetricTensor, SymmetryMappingSpec, SymDecompLinear
from arc25.vision2 import fields, attention, encoder, transformer, mae, swiglu, arc_solver
from arc25.training import saving, dataset, mae as mae_trainer, knn_eval, linear_probe, arc_solver as solver_trainer

In [2]:
os.environ["XLA_FLAGS"]="--xla_force_host_platform_device_count=2"
os.environ["EPATH_USE_TF"] = "false"

In [3]:
proj_root = Path("..").resolve()
data_root = proj_root / "data"
model_dir = data_root / "models"

In [4]:
src_dataset = dataset.ImagesDataset.load_compressed_cbor(
    data_root/"repack/re-arc.cbor.xz",
    filter=lambda iop,ex:  iop.input.shape == iop.output.shape,
)
challenge_order = tuple(sorted(src_dataset.challenges))
print(len(src_dataset.examples))

518152


In [5]:
reload(dataset)

small_eval = True

eval_split, train_split = src_dataset.split_by_challenge(
    np.random.default_rng(seed=42),
    n_min=10 if small_eval else 100,
)

input_ds, output_ds = train_split.split_input_output()

size_cuts = [12, 21, 30]
#size_cuts = [8,12,16,24,30]
#size_cuts = [30]

training_ds, = [dataset.BucketedDataset.make(
    s,
    set(itertools.product(size_cuts, size_cuts)) if s is not {} else [(30,30)],
    challenges=challenge_order,
) for s in [output_ds]]

for k,v in dict(
    training_ds=training_ds,
).items():
    print(f"{k}: { {kk:vv.n_examples for kk,vv in v.buckets.items()} }")


training_ds: {(30, 30): 35457, (21, 30): 69445, (21, 21): 39496, (12, 12): 21685, (12, 21): 47940, (12, 30): 43611}


In [6]:
reload(solver_trainer)

config = solver_trainer.ArcSolverConfig(
    seed = 42,
    
    batch_size = 16,
    ref_batch = 16,
    minibatch_size = 16,
    base_cell_cost = 10, 
        
    learning_rate = 1e-5,
    max_num_epochs = 1,
    max_num_ref_batches = 128,

    warmup_steps = 64,
    checkpoint_every_steps = 16,
    
    mode="flat",
    remat=True,
    unroll=None,

    eval_every_ref_batch = 2,
)



In [7]:
num_devices = jax.local_device_count()

minibatch_size_fn = dataset.MinibatchSizeFunction(
    reference_minibatch_size=config.minibatch_size,
    reference_image_size=config.reference_image_size,
    base_cost=config.base_cell_cost,
    granularity=num_devices,  # Ensure divisibility for pmap
)

batch_spec = dataset.BatchSpec(
    target_batch_weight=config.batch_size,
    reference_image_size=config.reference_image_size,
    # each image gets equal weight
    area_weight_exponent=None,
)

collator = dataset.BucketedCollator.make(
    dataset=training_ds,
    batch_spec=batch_spec,
    minibatch_size=minibatch_size_fn,
    seed=42, 
)

In [8]:
input_src = dataset.OnDemandBucketDataset(
    input_ds,
    bucket_shapes = tuple(sorted(training_ds.buckets.keys(),key=lambda sh:(sh[0]*sh[1],abs(sh[0]-sh[1])))),
    challenges = challenge_order,
    weight_fun = lambda area: None,
)

In [9]:
reload(arc_solver)

model_config = {
    k:v
    for k,v in mae.configs["tiny"].items()
    if k not in {"decoder_cell_infusion"}
}
model_config.update(
    num_program_tokens = 4,
    num_latent_programs = len(challenge_order),
)

solver = arc_solver.ARCSolver(
    **model_config,
    dtype=jnp.float32,
    rngs=nnx.Rngs(42),
)
width = solver.hidden_size

In [10]:
chkp_path = model_dir / "20251023-1137-vertex-ai-mae-tiny-4xL4-chkp-006912.msgpack.xz"
print(f"Loading encoder from checkpoint: {chkp_path}")
encoder_checkpoint = saving.load_model(chkp_path)
nnx.update(solver.encoder, encoder_checkpoint.state.model.encoder)
print("Encoder loaded successfully")

Loading encoder from checkpoint: /Users/yves/git-private/arc-2025/data/models/20251023-1137-vertex-ai-mae-tiny-4xL4-chkp-006912.msgpack.xz
Encoder loaded successfully


In [11]:
trainer = solver_trainer.ArcSolverTrainer.make(
    config=config,
    model=solver,
    collator=collator,
    inputs_src=input_src,
    eval_dataset = eval_split,
    num_devices = num_devices,
    rngs = nnx.Rngs(42),
)


In [12]:
trainer.with_progress_bars = True

In [13]:
stats = trainer.run_main()

--- ArcSolverTrainer ---
Run: 20251026-1016-ArcSolverTrainer
Devices: 2 × cpu
Training batch data weight: 16 (1 optimizer step)
Reference step data weight: 16 (~1.00 optimizer steps)
Total steps: 128
Evaluation: every 2 reference steps
----------------------------

Starting training...


Training:   0%|          | 0/128 [00:00<?, ?it/s]

Tracing _compute_grads for shape dict(cell_weight=(4,21,21), input_sizes=(4,2), inputs=(4,21,21), latent_program_idx=(4), output_masks=(4,21,21), outputs=(4,21,21)) (kw=dict(mode='flat', remat=True, unroll=None))
Tracing _compute_grads for shape dict(cell_weight=(5,12,30), input_sizes=(5,2), inputs=(5,12,30), latent_program_idx=(5), output_masks=(5,12,30), outputs=(5,12,30)) (kw=dict(mode='flat', remat=True, unroll=None))
Tracing _apply_update
Tracing _compute_grads for shape dict(cell_weight=(5,12,30), input_sizes=(5,2), inputs=(5,12,30), latent_program_idx=(5), output_masks=(5,12,30), outputs=(5,12,30)) (kw=dict(mode='flat', remat=True, unroll=None))
Tracing _compute_grads for shape dict(cell_weight=(2,30,30), input_sizes=(2,2), inputs=(2,30,30), latent_program_idx=(2), output_masks=(2,30,30), outputs=(2,30,30)) (kw=dict(mode='flat', remat=True, unroll=None))
Tracing _apply_update
Tracing _compute_grads for shape dict(cell_weight=(4,21,21), input_sizes=(4,2), inputs=(4,21,21), latent

  0%|          | 0/8 [00:00<?, ?it/s]

Tracing _embed_inputs for shape dict(input_sizes=(2,48,2), inputs=(2,48,12,12)) (kw=dict(mode='flat', remat=True, unroll=None, deterministic=True))
Tracing _embed_inputs for shape dict(input_sizes=(2,9,2), inputs=(2,9,12,21)) (kw=dict(mode='flat', remat=True, unroll=None, deterministic=True))
Tracing _embed_inputs for shape dict(input_sizes=(2,128,2), inputs=(2,128,12,21)) (kw=dict(mode='flat', remat=True, unroll=None, deterministic=True))
Tracing _embed_inputs for shape dict(input_sizes=(2,112,2), inputs=(2,112,12,30)) (kw=dict(mode='flat', remat=True, unroll=None, deterministic=True))
Tracing _embed_inputs for shape dict(input_sizes=(2,100,2), inputs=(2,100,21,21)) (kw=dict(mode='flat', remat=True, unroll=None, deterministic=True))
Tracing _embed_inputs for shape dict(input_sizes=(2,44,2), inputs=(2,44,21,30)) (kw=dict(mode='flat', remat=True, unroll=None, deterministic=True))
Tracing _embed_inputs for shape dict(input_sizes=(2,128,2), inputs=(2,128,21,30)) (kw=dict(mode='flat', rema

...

Evaluation completed in 62.4s: cell_accuracy: 0.722 loss: 2.061 pair_accuracy: 0.003 class_accuracy_histogram: [261.000,0.000,2.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000]
Tracing _compute_grads for shape dict(cell_weight=(2,21,30), input_sizes=(2,2), inputs=(2,21,30), latent_program_idx=(2), output_masks=(2,21,30), outputs=(2,21,30)) (kw=dict(mode='flat', remat=True, unroll=None))

[Step 4] Running evaluation...


  0%|          | 0/8 [00:00<?, ?it/s]

Evaluation completed in 44.8s: cell_accuracy: 0.722 loss: 2.058 pair_accuracy: 0.003 class_accuracy_histogram: [261.000,0.000,2.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000]
Tracing _compute_grads for shape dict(cell_weight=(12,12,12), input_sizes=(12,2), inputs=(12,12,12), latent_program_idx=(12), output_masks=(12,12,12), outputs=(12,12,12)) (kw=dict(mode='flat', remat=True, unroll=None))

[Step 6] Running evaluation...


  0%|          | 0/8 [00:00<?, ?it/s]


Training interrupted by user

--- Training Finished ---
Total time: 544.9s
Average throughput: -0.3 weight/s


In [14]:
cache = trainer._eval_data_cache
print(cache.total_weight, cache.per_class_total_weight.sum())
print(cache.per_class_total_weight)
print(sum(w.sum() if (w:=mb.get("weight")) is not None else mb["latent_program_idx"].size for mb in cache.minibatches))


1306 1306
[0 5 0 5 5 0 5 5 4 5 5 5 5 0 5 5 5 5 0 5 0 0 5 5 5 0 5 5 0 5 0 5 5 5 5 0 5
 0 0 5 5 5 5 5 5 0 5 0 0 5 5 5 5 5 5 0 0 5 5 5 5 5 5 5 0 5 0 5 5 5 5 0 5 5
 5 5 5 5 0 5 5 5 0 5 5 5 5 0 5 5 0 5 5 5 5 0 5 5 5 0 5 5 0 0 5 0 0 0 0 5 0
 5 5 0 0 0 5 5 5 5 0 5 0 0 5 5 5 5 5 0 5 5 5 0 0 5 5 0 5 5 5 0 5 0 5 0 5 5
 0 5 5 0 0 5 5 5 5 5 0 5 4 5 5 0 5 5 5 5 5 0 5 0 5 0 5 5 0 0 5 0 5 5 0 0 0
 0 5 0 0 5 5 5 5 0 0 5 5 5 5 5 0 5 5 5 0 5 0 5 0 0 0 5 0 5 5 0 5 0 5 5 0 5
 0 5 5 5 0 5 5 5 0 5 0 4 0 0 5 0 0 5 5 0 5 0 5 5 0 5 0 5 5 5 0 5 5 5 0 5 0
 5 5 5 0 0 5 5 5 5 0 5 0 5 5 0 0 5 5 5 5 5 5 5 5 5 5 5 5 5 0 0 0 5 5 5 0 0
 5 5 5 0 5 5 5 0 5 5 0 0 5 0 0 5 5 5 0 0 5 0 0 5 0 5 5 5 2 0 0 5 5 5 5 5 5
 1 5 5 5 5 0 5 5 5 5 5 5 0 0 5 5 5 0 4 5 5 0 5 5 5 4 0 5 5 5 5 0 0 5 5 5 5
 5 0 5 5 5 0 0 5 5 5 5 5 5 3 5 0 5 0 5 5 0 5 0 0 0 0 5 0 0 0]
1306
